In [13]:
import pandas as pd
import numpy as np
import sklearn as sk
import altair as alt

In [14]:
nba_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_train.csv')
nba_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_test.csv')

In this notebook we will be fitting a logistic regression model with an L1 penalty to encourage coefficient sparsity.

# Model

First we can describe the model of interest under a statistical framework. Denote the following quantities:
1) y: an n x 1 vector, containing the binary variable of interest
2) X: and n x (p+1) matrix, consisting of the feature variables and intercept
3) $\beta$: A (p+1) x 1 vector of coefficients.

We will also use the following functions:

1) $logit(p) = log(\frac{p}{1-p})$; this is often denoted as the log-odds
2) $expit(x) = \frac{1}{1+exp(-x)}$; this is the inverse function of logit, i.e. $expit(x) = logit^{-1}(x)$

The model we will be utilizing is:
$$
y_i \sim Bernoulli(p_i = expit(x_i^{T}\beta))
$$

Where $x_i$ is the i'th row of the feature matrix X.

If we assume independence (clearly broken here since player performance is clearly correlated across different seasons, but we will disregard this for now), then we have a likelihood function of the form:

$$
L(\beta) = \prod_{i=1}^n (expit(x_i^{T}\beta))^{y_i}\times (1-expit(x_i^{T}\beta))^{1-y_i}
$$

Leading to a log-likelihood function (our unregularized negative objective function) of:

$$
\ell(\beta) = \sum_{i=1}^n y_i(log(expit(x_i^{T}\beta))) + (1-y_i)log(1-expit(x_i^{T}\beta)) \\ = \sum_{i=1}^n y_i(x_i^T\beta)-log(1+exp(x_i^T\beta))
$$



Thus we will be finding:

$$
\underset{\beta}{min}\sum_{i=1}^n -y_i(x_i^T\beta)+log(1+exp(x_i^T\beta)) + r(\beta)
$$

where $r(\beta)$ is a regularization term

For the L1 regularizer SKlearn specifically will be minimizing:
$$
\underset{\beta}{min} \ C\sum_{i=1}^n -y_i(x_i^T\beta)+log(1+exp(x_i^T\beta)) + \sum_{i=0}^p|\beta_i|
$$

Where the regularizing constant is given as C>0. This is a hyperparameter we must tune

# Fitting Logistic Model to NBA Data

Now we can begin to apply this model to our data.

First we can examine the proportions of All-NBA in our dataset. We initially created a train-test split where we attempt to create similar distributions for each set.

For the training set we have:

In [15]:
#Proportions of all_nba_c_year
nba_train['all_nba_c_year'].value_counts(normalize=True)

0    0.967255
1    0.032745
Name: all_nba_c_year, dtype: float64

For the testing stat we have:

In [16]:
nba_test['all_nba_c_year'].value_counts(normalize=True)

0    0.96727
1    0.03273
Name: all_nba_c_year, dtype: float64

Both sets have similar proportions as expected, but clearly we have an incredibly unbalanced dataset, but we may try to fit our models initially to see if we can improve upon them.

### Data Filtering

First we may try to filter our data based on variables like minutes played (`MP`), games played (`G`), since we know that these awards go to the best players in the NBA, and the best players tend to play a lot. In the new 2023 CBA (collective bargaining agreement), there is a minimum game requirement (65 games) that must be met in order to win All-NBA. However, since this rule was not in place for prior awards (where this data comes from), we can instead filter so that we only consider players who have played more games than the players with the least minutes played and games played that still won All-NBA.

In [17]:
min_minutes = nba_train[(nba_train['all_nba_c_year']==1)].MP.min()
min_G = nba_train[(nba_train['all_nba_c_year']==1)].G.min()
nba_filt_train = nba_train[(nba_train['MP']>=min_minutes) & (nba_train['G']>=min_G)]
nba_filt_test = nba_test[(nba_test['MP']>=min_minutes) & (nba_test['G']>=min_G)]




y_train = nba_filt_train['all_nba_c_year']

y_test = nba_filt_test['all_nba_c_year']

First we may fit a simpler model. Which players will make all-nba, versus which players won't? In this case we will ignore teams and instead only focus on the binary indicator. We will also see how the classifier does when we don't filter for positions

We will construct this model using a current year players stats and predict whether they make all_nba in the current year.

### Fitting the Model

Since we have already standardized our variables, we must simply one hot encode the categorical variable, `Tm`. We will use a pipeline for ease of use.

We will consider the following parameter grid for C. We will select the best C based off of k-fold cross validation, with k=5.

In [57]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

num_features = ['Age','G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'W',
       'num_all_nba']

cat_features = ['Tm']

#Now I will create a pipeline where I extract my_features, and apply OHE to cat_features
ct = ColumnTransformer(
    [("select", "passthrough", num_features),
     ("ohe", OneHotEncoder(handle_unknown="ignore"), cat_features)],
     remainder="drop"
)



clf = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l2', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])

#consider grid of C values between 0 and 1. Larger interval was 
#checked first, and then the interval was narrowed down to 0 to 1
param_grid2 = {'classifier__C': np.arange(0.01, 1.01, 0.01)}
model = GridSearchCV(clf, param_grid2)

model.fit(nba_filt_train, y_train)

# Results

We now have the following metrics for our model.

In [51]:
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(nba_filt_test)))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97       819
           1       0.81      0.74      0.77       123

    accuracy                           0.94       942
   macro avg       0.89      0.86      0.87       942
weighted avg       0.94      0.94      0.94       942



In [53]:
model.best_params_

{'classifier__C': 0.15}

In [56]:
model.cv_results_.items()

dict_items([('mean_fit_time', array([0.01442361, 0.02235103, 0.02797999, 0.02222881, 0.03539066,
       0.03637457, 0.06073017, 0.08284574, 0.07104864, 0.09200077,
       0.06417117, 0.10403972, 0.10013118, 0.10272603, 0.13682137,
       0.11112041, 0.12752604, 0.10954585, 0.12594862, 0.14090967,
       0.14731917, 0.10359397, 0.11989121, 0.13967462, 0.14225078,
       0.13397398, 0.13095846, 0.13387766, 0.16000485, 0.15493932,
       0.18220887, 0.21690211, 0.20321288, 0.22082925, 0.20778275,
       0.23309355, 0.32452822, 0.27278118, 0.25449224, 0.31603451,
       0.26720324, 0.31058693, 0.33655229, 0.33177915, 0.29757333,
       0.35081921, 0.34025154, 0.34134803, 0.33378978, 0.31803379,
       0.35030961, 0.32943087, 0.3618618 , 0.351261  , 0.33538704,
       0.31403775, 0.35411634, 0.30192242, 0.31864042, 0.3626307 ,
       0.31308365, 0.3162643 , 0.32155418, 0.31866269, 0.33440118,
       0.29653172, 0.32236867, 0.32921944, 0.32060242, 0.32307625,
       0.3262414 , 0.32931933, 0

We see that our precision is .82, indicating that of the model's predicted positives, 82% of them were correct. The recall, of .75, indicates only 75% of the All-NBA players in the dataset were classified as All-NBA.  

Looking at the confusion matrix for these results we see that we had 20 false postives, and 31 false negatives. These are not ideal, but considering the size of our dataset, may still work for our purposes.

In [20]:
pd.crosstab(y_test, model.predict(nba_filt_test), rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,799,20
1,32,91


We also see that our L1 penalty reduced the number of covariates from 79 to 34 

In [21]:
#Now we will extract the feature names from the pipeline
feature_names = model.best_estimator_.named_steps['col_transform'].get_feature_names_out()
coef_df = pd.DataFrame({'coef':model.best_estimator_['classifier'].coef_[0]
                        ,'var':feature_names})
coef_df.shape

(79, 2)

In [22]:
coef_df_nz = coef_df[coef_df['coef']!=0]
coef_df_nz.shape

(31, 2)

We see from our coefficient plot that FGA seems to have the highest impact on the log-odds of winning All-NBA, while FT% has the biggest negative impact. Surprisingly we also see that usage % is another of the strongest negative covariates. This bears further study, but for the purposes of this project (prediction), we do not necessarily care about how the model is weighing each of the features

In [23]:
#Now we will make a bar chart of these coefficients
alt.Chart(coef_df_nz).mark_bar().encode(
    y=alt.Y('coef',title='Coefficient'),
    x=alt.X('var',title='Variable', sort = '-y'))

alt.Chart(...)

We see for this dataset we have the following players who were predicted All-NBA but did not win it. (False Positives)

In [24]:
nba_filt_test[(model.predict(nba_filt_test)==1) & (y_test!=1)][['Player',"year"]]

,Player,year
463,Ja Morant,2023
514,Ben Simmons,2019
577,James Harden,2023
794,Trae Young,2021
851,James Harden,2021
1152,Chauncey Billups,2008
1803,Horace Grant,1992
1951,Andrei Kirilenko,2004
1995,Karl Malone,2003
2285,Josh Smith,2012


For our False Negatives we have:

In [25]:
nba_filt_test[(model.predict(nba_filt_test)!=1) & (y_test==1)][['Player',"year"]]

,Player,year
23,Gary Payton,1994
284,Joe Dumars,1990
304,Joe Dumars,1993
314,Mitch Richmond,1996
652,Mitch Richmond,1997
717,Isiah Thomas,1987
1010,Chauncey Billups,2009
1055,Tim Hardaway,1993
1079,Manu Ginóbili,2011
1093,Stephon Marbury,2003


Now we may consider fitting position specific models. We will use the same `MP` and `G` filters we used prior.

In [26]:
nba_g_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_g_train.csv')
nba_g_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_g_test.csv')

In [27]:
nba_g_filt_train = nba_g_train[(nba_g_train['MP']>=min_minutes) & (nba_g_train['G']>=min_G)]
nba_g_filt_test = nba_g_test[(nba_g_test['MP']>=min_minutes) & (nba_g_test['G']>=min_G)]
y_g_train = nba_g_filt_train['all_nba_c_year']
y_g_test = nba_g_filt_test['all_nba_c_year']


In [28]:
#Now I will create my pipeline as before, but I will include a step to remove a subset of variables I specify
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


clf_g = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])

model_g = GridSearchCV(clf_g, param_grid)
model_g.fit(nba_g_filt_train, y_g_train)

GridSearchCV(estimator=Pipeline(steps=[('col_transform',
                                        ColumnTransformer(transformers=[('select',
                                                                         'passthrough',
                                                                         ['Age',
                                                                          'G',
                                                                          'GS',
                                                                          'MP',
                                                                          'FG',
                                                                          'FGA',
                                                                          'FG%',
                                                                          '3P',
                                                                          '3PA',
                                                                          '3P%',
                                                                          '2P',
                                                                          '2PA',
                                                                          '2P%',
                                                                          'eFG%',
                                                                          'FT',
                                                                          'FTA',
                                                                          'FT%',
                                                                          'ORB',
                                                                          'DRB',
                                                                          'TRB',
                                                                          'AST',
                                                                          'STL',
                                                                          'BLK',
                                                                          'TOV',
                                                                          'PF',
                                                                          'PTS',
                                                                          'PER',
                                                                          'TS%',
                                                                          '3PAr',
                                                                          'FTr', ...]),
                                                                        ('ohe',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Tm'])])),
                                       (...',
                                        LogisticRegression(max_iter=10000,
                                                           penalty='l1',
                                                           random_state=0,
                                                           solver='liblinear'))]),
             param_grid={'classifier__C': array([1.00000000e-04, 3.72759372e-04, 1.38949549e-03, 5.17947468e-03,
       1.93069773e-02, 7.19685673e-02, 2.68269580e-01, 1.00000000e+00,
       3.72759372e+00, 1.38949549e+01, 5.17947468e+01, 1.93069773e+02,
       7.19685673e+02, 2.68269580e+03, 1.00000000e+04])})

By looking at only guards we have similar accuracy, with worse recall for true positives

In [29]:
print(classification_report(y_g_test, model_g.predict(nba_g_filt_test)))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       369
           1       0.82      0.65      0.73        49

    accuracy                           0.94       418
   macro avg       0.89      0.82      0.85       418
weighted avg       0.94      0.94      0.94       418



In [30]:
pd.crosstab(y_g_test, model_g.predict(nba_g_filt_test), rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,362,7
1,17,32


We have our false positives as:

In [31]:
nba_g_filt_test[(model_g.predict(nba_g_filt_test)==1) & (y_g_test!=1)][['Player','year']]

,Player,year
463,Ja Morant,2023
577,James Harden,2023
774,Jimmy Butler,2015
851,James Harden,2021
1152,Chauncey Billups,2008
1187,Kevin Johnson,1997
1250,John Stockton,2000


We have our false negatives as:

In [32]:
nba_g_filt_test[(model_g.predict(nba_g_filt_test)!=1) & (y_g_test==1)][['Player','year']]

,Player,year
23,Gary Payton,1994
284,Joe Dumars,1990
304,Joe Dumars,1993
314,Mitch Richmond,1996
652,Mitch Richmond,1997
673,Deron Williams,2010
717,Isiah Thomas,1987
1010,Chauncey Billups,2009
1055,Tim Hardaway,1993
1079,Manu Ginóbili,2011


Examining our coefficients we see the L1 penalty reduced our number of features by almost 60.

In [33]:
coef_g_df = pd.DataFrame({'coef':model_g.best_estimator_['classifier'].coef_[0],
                          'var':feature_names})
coef_df.shape

(79, 2)

In [34]:
coef_g_nz = coef_g_df[coef_g_df['coef']!=0]
coef_g_nz.shape

(17, 2)

Looking at their values we have:

In [35]:
alt.Chart(coef_g_nz).mark_bar().encode(
    y=alt.Y('coef',title='Coefficient'),
    x=alt.X('var',title='Variable', sort = '-y'))

alt.Chart(...)

We see here that VORP has one of the largest positive coefficients, while number of personal fouls is one of the largest magnitude negative coefficients. 

Repeating this analysis for the C's and F's we get:

F's:

In [36]:
nba_F_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_F_train.csv')
nba_F_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_F_test.csv')

nba_F_filt_train = nba_F_train[(nba_F_train['MP']>=min_minutes) & (nba_F_train['G']>=min_G)]
nba_F_filt_test = nba_F_test[(nba_F_test['MP']>=min_minutes) & (nba_F_test['G']>=min_G)]


y_F_train = nba_F_filt_train['all_nba_c_year']
y_F_test = nba_F_filt_test['all_nba_c_year']

In [37]:
clf_F = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_F = GridSearchCV(clf_F, param_grid)
model_F.fit(nba_F_filt_train, y_F_train)

KeyboardInterrupt: 

In [ ]:
print(classification_report(y_F_test, model_F.predict(nba_F_filt_test)))

In [ ]:
pd.crosstab(y_F_test, model_F.predict(nba_F_filt_test), rownames=["Actual"], colnames=["Predicted"])

In [ ]:
coef_F_df = pd.DataFrame({'coef':model_F.best_estimator_['classifier'].coef_[0],
                          'var':feature_names})
coef_F_df.shape

In [ ]:
coef_F_nz = coef_F_df[coef_F_df['coef']!=0]
coef_F_nz.shape

In [ ]:
alt.Chart(coef_F_nz).mark_bar().encode(
    y=alt.Y('coef',title='Coefficient'),
    x=alt.X('var',title='Variable', sort = '-y'))

C's:

In [ ]:
nba_C_train = pd.read_csv('Data_Scripting_Cleaning/Full_data/Training_Sets/nba_C_train.csv')
nba_C_test = pd.read_csv('Data_Scripting_Cleaning/Full_data/Test_Sets/nba_C_test.csv')

nba_C_filt_train = nba_C_train[(nba_C_train['MP']>=min_minutes) & (nba_C_train['G']>=min_G)]
nba_C_filt_test = nba_C_test[(nba_C_test['MP']>=min_minutes) & (nba_C_test['G']>=min_G)]


y_C_train = nba_C_filt_train['all_nba_c_year']
y_C_test = nba_C_filt_test['all_nba_c_year']

clf_C = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_C = GridSearchCV(clf_C, param_grid)
model_C.fit(nba_C_filt_train, y_C_train)

We see the Centers had a quite poor positive recall in this dataset. This makes sense, since this dataset is the most unbalanced of the three, since only one center is picked for each All-NBA team.

In [ ]:
print(classification_report(y_C_test, model_C.predict(nba_C_filt_test)))

In [ ]:
pd.crosstab(y_C_test, model_C.predict(nba_C_filt_test), rownames=["Actual"], colnames=["Predicted"])

In [ ]:
coef_C = pd.DataFrame({'coef':model_C.best_estimator_['classifier'].coef_[0],
                       'var':feature_names})
coef_C.shape

In [ ]:
coef_C_nz = coef_C[coef_C['coef']!=0]
coef_C_nz.shape

In [ ]:
alt.Chart(coef_C_nz).mark_bar().encode(
    y=alt.Y('coef',title='Coefficient'),
    x=alt.X('var',title='Variable', sort = '-y'))

For this model Wins and the advanced statistic Win shares were two of the stats with the largest positive impact on the model, while FT% was one of the largest negative coefficients

Now we may try to balance our datasets. First we will try this on the larger dataset. We will first use SMOTENC sampling (Synthetic Minority Oversampling Technique-Numerical, Categorical). This method uses a k-nearest neighbors approach (default k=5).

In [ ]:
#balance all_nba_c_year in the nba_train set
from imblearn.over_sampling import SMOTENC

smote = SMOTENC(random_state=0,categorical_features=[nba_filt_train[num_features+cat_features].shape[1]-1])
X_train_resampled, y_train_resampled = smote.fit_resample(nba_filt_train[num_features+cat_features], nba_filt_train['all_nba_c_year'])

In [ ]:
clf_bal = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_smote = GridSearchCV(clf_bal, param_grid)
model_smote.fit(X_train_resampled, y_train_resampled)

In [ ]:
print(classification_report(nba_filt_test['all_nba_c_year'], model_smote.predict(nba_filt_test[num_features+cat_features])))

In [ ]:
pd.crosstab(nba_filt_test['all_nba_c_year'], model_smote.predict(nba_filt_test[num_features+cat_features]), rownames=["Actual"], colnames=["Predicted"])

We see using this balanced dataset greatly increases our recall, but at the cost of our precision. 

We may also try under-sampling. This will essentially remove rows of the majority class (those who did not make All-NBA).

In [ ]:
#Now we may use undersampling to balance the classes
from imblearn.under_sampling import RandomUnderSampler
ran_uns = RandomUnderSampler(random_state=0)
X_train_resampled, y_train_resampled = ran_uns.fit_resample(nba_filt_train[num_features+cat_features], nba_filt_train['all_nba_c_year'])


In [ ]:
clf_us_bal = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_us = GridSearchCV(clf_us_bal, param_grid)
model_us.fit(X_train_resampled, y_train_resampled)

In [ ]:
print(classification_report(nba_filt_test['all_nba_c_year'], model_us.predict(nba_filt_test[num_features+cat_features])))

In [ ]:
pd.crosstab(nba_filt_test['all_nba_c_year'], model_us.predict(nba_filt_test[num_features+cat_features]), rownames=["Actual"], colnames=["Predicted"])

Finally we can try over-sampling.

In [ ]:
#Now we may use oversampling to balance the classes
from imblearn.over_sampling import RandomOverSampler
ran_os = RandomOverSampler(random_state=0)
X_train_resampled, y_train_resampled = ran_os.fit_resample(nba_filt_train[num_features+cat_features], nba_filt_train['all_nba_c_year'])
clf_os_bal = Pipeline([
    ("col_transform", ct),
    ("classifier", LogisticRegression(penalty = 'l1', solver = 'liblinear', 
                                      max_iter = 10000, random_state=0))
])
model_os = GridSearchCV(clf_os_bal, param_grid)
model_os.fit(X_train_resampled, y_train_resampled)

In [ ]:
print(classification_report(nba_filt_test['all_nba_c_year'], model_os.predict(nba_filt_test[num_features+cat_features])))

In [ ]:
pd.crosstab(nba_filt_test['all_nba_c_year'], model_os.predict(nba_filt_test[num_features+cat_features]), rownames=["Actual"], colnames=["Predicted"])

We see all of these 3 methods are not significantly better, or are quite worse than the baseline unbalanced class model. Specifically we have quite poor precision for our positive classifications 